In [11]:
#!pip install pytesseract
#!pip install selenium
#!pip install openai

In [1]:
import os
from apikey import YOUR_API_KEY

In [2]:
def AIopinion(messages):
    from openai import OpenAI
    
    import re
    from IPython.display import clear_output
    
    client = OpenAI(api_key=YOUR_API_KEY, base_url="https://api.perplexity.ai")
    
    try:
        response_stream = client.chat.completions.create(
            model="sonar-pro", #sonar-pro #sonar-reasoning-pro #sonar #sonar-reasoning
            messages=messages,
            stream=True,
        )
    
        full_response = []
        link_map = {}
        link_counter = 1
        buffer = ""
    
        for chunk in response_stream:
            if hasattr(chunk, 'citations') and chunk.citations:
                # Enumerate URLs starting at index 1 for proper citation numbering
                link_map = {str(i): url for i, url in enumerate(chunk.citations, start=1)}
            if chunk.choices and chunk.choices[0].delta.content:
                content = chunk.choices[0].delta.content
                buffer += content
                
                # Process URLs incrementally
                while True:
                    match = re.search(r'https?://[^\s\)\]\}]+', buffer)
                    if not match:
                        break
                        
                    url = match.group()
                    if url not in link_map:
                        link_map[url] = link_counter
                        link_counter += 1
                    
                    # Replace URL with reference marker
                    buffer = buffer.replace(url, f'[{link_map[url]}]', 1)
                
                # Print processed content and clear buffer
                print(buffer, end='', flush=True)
                full_response.append(buffer)
                buffer = ""
    
        # Print remaining buffer content
        if buffer:
            print(buffer, end='', flush=True)
            full_response.append(buffer)
    
        # Add references appendix
        print("\n\n--- References ---")
        for url, num in sorted(link_map.items(), key=lambda x: x[1]):
            print(f"[{num}] {url}")
    
    except Exception as e:
        clear_output()
        print(f"Error occurred: {str(e)}")

In [3]:
from PIL import Image
import pytesseract  # First install: pip install pytesseract AND tesseract-OCR

# Convert image to analyzable text
def image_to_analysis(image_path):
    image = Image.open(image_path)
    text_data = pytesseract.image_to_string(image)
    
    # Clean up OCR results
    cleaned_data = "\n".join([line for line in text_data.split('\n') if line.strip()])
    return cleaned_data

In [4]:
def persist_chart_for_analysis(url, image_path):
    from selenium import webdriver
    from selenium.webdriver.chrome.options import Options

    options = Options()
    options.add_argument("--headless")  # Run in background
    driver = webdriver.Chrome(options=options)

    driver.get(url)
    driver.implicitly_wait(15)  # Wait for chart to load

    # Take full-page screenshot
    driver.save_screenshot(image_path)
    driver.quit()
    

In [14]:
image_path = 'chart.png'
#url = "https://www.tradingview.com/script/Edd4Wt4b-My-Momentum-Strategy/"
url = "https://s.tradingview.com/widgetembed/?symbol=NASDAQ:AAPL&interval=1D&theme=dark&studies=RSI@tv-basicstudies,MACD@tv-basicstudies"

In [15]:
persist_chart_for_analysis(url=url, image_path=image_path)
processed_data = image_to_analysis(image_path)

In [16]:
print(processed_data)

268.00
~
il |
WM
beer eerie eee leer cere ere neler eee er eee erected ee eee A | (a
Aw of / | 1 vou
hy hte wt IY yl Yl —
Py yy yi hi
, { ie
i ~
w “>


In [8]:
messages = [{
    "role": "user",
    "content": f"""Analyze this data:
    
    ## 
    {processed_data}
    """
}]

In [12]:
AIopinion(messages=messages)

Based on the search results, here's an analysis of momentum trading strategies:

Momentum trading is a strategy that capitalizes on the continuation of existing trends in asset prices. The key principles of momentum trading include:

1. Identifying assets with strong price movements in a particular direction (upward or downward)[1][5].

2. Entering positions to profit from the momentum continuing in the same direction[1].

3. Using technical analysis tools and indicators like moving averages, relative strength index (RSI), and price momentum oscillators to identify securities with strong momentum trends[1][3].

Key aspects of momentum trading strategies:

1. Time frames: Momentum strategies can be applied to various time frames, from short-term (days or weeks) to longer-term (months)[3][5].

2. Asset selection: Traders often focus on a universe of stocks or ETFs, such as the S&P 500[4][8].

3. Ranking and selection: Assets are typically ranked based on their price momentum over specifi